In [17]:
import csv
import pandas as pd
import numpy as np

with open("test.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['t', 'r_eci_x', 'r_eci_y', 'r_eci_z', 'v_eci_x', 'v_eci_y', 'v_eci_z', 
                     'q_BI_x', 'q_BI_y', 'q_BI_z', 'q_BI_w', 'omega_x', 'omega_y', 'omega_z', 
                     'omega_rw', 'tau_rw', 'i_mag'])
    
    for i in range(10):
        writer.writerow([i, *np.array((1, 2, 3), dtype=float), *np.array((1, 2, 3)), *np.array((1, 2, 3, 4)), *np.array((1, 2, 3)), 0.001, 12, 14])

df = pd.read_csv("test.csv")
for col in df.columns:
    if df[col].dtype == "object":
        print(type(df[col][0]))
        

In [20]:

sum([np.array((-10, i)) for  i in range(3)])

array([-30,   3])

In [55]:
import datetime
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time


    
time = Time(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), scale="utc", format="iso")
sun = coord.get_sun(time)

sun.cartesian.xyz.to(u.m).value

array([-7.26797845e+10, -1.18021010e+11, -5.11600340e+10])

In [21]:
import numpy as np
class Surface:
    def __init__(self, center: np.ndarray, normal: np.ndarray, width: float, height: float, u: np.ndarray, v: np.ndarray):
        self.center = center
        self.normal = normal / np.linalg.norm(normal)
        self.width = width
        self.height = height
        self.u = u / np.linalg.norm(u)
        self.v = v / np.linalg.norm(v)
        self.area = width * height
    
    def self_occlusion(self, ray: np.ndarray, surfaces: list["Surface"]) -> bool:
        
        for s in surfaces:
            if s == self:
                continue
            
            if s.passes_through(self.center, ray):
                return True
        
        return False
    
    def passes_through(self, point: np.ndarray, direction: np.ndarray) -> bool:
        
        dn = np.dot(direction, self.normal)
        
        # ray is parallel
        if dn == 0:
            return False
        
        t = np.dot(self.center - point, self.normal) / dn

        if t >= 0:
            return False


        print(t)

        p = point + t * direction

        print(p)

        p_xy = np.vstack([self.u, self.v]) @ (p - self.center)
        
        return bool(np.all(np.abs(p_xy) <= np.array([self.width, self.height]) / 2))



        

s = Surface(np.array([0, 0, 0]), [0, 0, 1], 20, 40, [1, 0, 0], [0, 1, 0])

s.passes_through(np.array([0, 0, 15]), np.array([1, 0, -1]))


def center_of_pressure(surfaces: list["Surface"]) -> np.ndarray:
    """
    Calculates the geometric center of pressure for a collection of surfaces.

    This is a simplified model that computes the area-weighted average of the
    centers of the provided surfaces.

    Parameters
    ----------
    surfaces : List[Surface]
        A list of Surface objects.

    Returns
    -------
    np.ndarray, shape (3,)
        The calculated center of pressure vector [m].
    """

    # TODO: idk need a better center of pressure formula
    
    A_tot = sum([s.area for s in surfaces])

    cop = sum([s.area * s.center for s in surfaces]) / A_tot

    return cop

center_of_pressure([s, s, s])

array([0., 0., 0.])

In [27]:
from scipy.spatial.transform import Rotation as R
import numpy as np

r = R.from_quat([0.76, 0.14, 0, 0], scalar_first=True)

r.apply(np.array([1, 2, 3]))

array([1.        , 0.79973208, 3.51574012])

In [28]:
print(1362/299792458)

4.543142976598831e-06
